In [1]:
!nvidia-smi

Sun Jun 12 03:11:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers pandas sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 57.2 MB/s 
     |████████████████████████████████| 6.6 MB 49.5 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machi

In [8]:
from pprint import pprint
import random
from datetime import datetime
from sklearn.model_selection import train_test_split

import re
import pandas as pd
from transformers import AutoModelForMaskedLM,AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments

In [10]:
train_df = pd.read_csv('train.csv')

In [11]:
train_df.head(5)

,id,flag,title,content
0,3701be69fdea4881874d073b8cc3bfdb,1,办理了ETC高速浪一下，一出门就遇到阿尔法，100万哪，老款的我坐过也开过，确实好车，但是没...,办理了ETC高速浪一下，一出门就遇到阿尔法，100万哪，老款的我坐过也开过，确实好车，但是没...
1,967b683e4742489fb4738eb7d5f0ae4e,0,秋介绍~,秋介绍~
2,f98f7ea52a9b460dbc7d32b4114b925b,1,"""精""改埃尔法提车记！",因为手上的两地牌关系，一直都喜欢买跑车开，觉得用了两地牌在国内开，而且完善的赛道设施，...
3,8cce3056ad9b453996f22508cf5e1d28,0,?里有卖？怎么联系？,?里有卖？怎么联系？
4,0835ce6f7fcb4c7b96490b65f66a65f9,1,“宜”与“亦”之间的区别？这辆车可以回答,灵山清净，拈花一笑。没想到这次的试驾地竟是如此禅意：太湖...


In [12]:
train_sentences = train_df['content'].tolist()

In [13]:
train_sentences[:10]

['办理了ETC高速浪一下，一出门就遇到阿尔法，100万哪，老款的我坐过也开过，确实好车，但是没感觉值那么多，中国豪人太多。 走磨合测试定速巡航 父亲没空就带着家里的各位老板们兜风  途中也感受下老板位舒服不，辛苦大老板了，舒适版有个好处，可以不用下车就可以在车内完成换司机的任务，这个对于下雨天啥的，非常实用。  三老板，其实是最大的老板，哭起来，我都要投降 二老板  趁着假装老板的机会好好感受下车内，第2排左侧位置放个纸巾不错 中间没有扶手的好处可以放相机包还有垃圾桶，这个建议大家都要备一个，乱丢垃圾的事我们可不干。 2排空调面板设计的还不错 扶手也挺好的，多有 电镀处理，而且是亚光的，更显质感。  一路奔袭，这条路上基本无车，风景又美，舒服惬意。 终于抵达目的地，永泰的葱饼店，这个可是福州第一名的葱饼，高峰期的时候要排队一个小时，今天时候尚早还没开始排，赶紧趁热走一个  途进服务区，拍个造型  这个角度有没很帅 最后大鹏展翅来一个  来回50多公里吧，平均油耗6.7.80公里时速，这个成绩这样的车重应该算是优秀的。  目前使用了200多公里，感受良好，全家一直满意，现在有点后悔没有一步到位上个顶配。算了，走改装之路吧！比较遗憾的是目前还没找到电动后视镜，还有自动刹车也无法加装，其他的配置倒不是很在意。',
 '秋介绍~',
 ' \u3000\u3000因为手上的两地牌关系，一直都喜欢买跑车开，觉得用了两地牌在国内开，而且完善的赛道设施，可以在赛道上飚车确实在香港是无法想象的 。  \u3000\u3000因为年纪渐长，有了家庭，为了能够方便孩子家里出入方便，决定换一台7人保姆车来开，近期埃尔法在国内流行人气指数又超高，于是决定入手这辆适合家庭出行的保姆车，开来看去原厂样式配置的埃尔法就是有点老气的感觉，所以一直在看日本改装大厂对这台车的改装有哪些自己比较喜欢的，结果一看到Wald对这台车的改装后立即就吸引到我了，拿起电话打给中国总代，问问关于这台车的改装方案。   ',
 '?里有卖？怎么联系？',
 '                  灵山清净，拈花一笑。没想到这次的试驾地竟是如此禅意：太湖畔，灵山旁。闲庭信步于古朴木石的小镇，目之所及是随风摇曳的花草，暖阳斜撒，鸟语风铃，戏台钟鼓，流水观佛，竟有种穿梭至古时之唐朝，今日之京都感。细细打听，原来这里早被世界

In [ ]:
val_df = pd.read_csv('test.csv')
val_sentences = val_df['content'].tolist()

val_sentences[:10]

['mb 现在都加价 ',
 '既然有这个加装的想法,你是可以去当地的专业汽车改装装饰店看看的,多走几家对比了解一下哪家好。',
 '已经关注了价格多少',
 '，额色222色啊啊啊-.饿死我我晚上 哇数学，，，，，众互相学习那我问问色鬼v我问问色巫毒娃娃点题vvvv斧头帮是ds rcz7787*787857774休息休/#,#8息休息休息休息不xcszsfv 这是谁的伤人心啊7_自行车VC小',
 '\u3000\u3000一般我们买车是因为有这个需求，考虑到家庭人员多，对车子乘坐空间和舒适性上，都是有比较高要求的。又加上自己做点小生意，生意往来有时候需要接待客户，所以最终决定买MPV的。在决定选择商务MPV的时候，也看过其他车型，但是最后是舒朗雅致的全新一代GL8征服了我。其实一开始只是想先看看而已，后来就当是去车展上多见识些车型，多了解这个市场而已。打算要是没有特别中意的就明年再买车。我原先在车展上看到的那些都是家轿，不然就是SUV，鲜少看到有MPV。最后还是偶然间看到了别克全新一代GL8，就被吸引住了。外观优雅，却是自带强大气场。觉得这样的豪华MPV，应该就是我想要的车型了。如今了解过后，真正入手GL8，希望生意会顺风顺水，发发发！  \u3000\u3000GL8是别克的招牌车型，也是市场上的传奇。一直都是MPV界的销量冠军，非常受欢迎。我提的是17款全新一代GL8旗舰型，整车造型感觉很好。  \u3000\u3000展翼型的进气格栅，立体动感。这个角度看，感受的更清楚。  \u3000\u3000GL8的造型，可以说是真的很年轻动感，但是又有家居气质。是一辆宜家宜商的“百搭”车型！  \u3000\u3000别克的LOGO，依然亮眼。  \u3000\u3000车身侧面看起来比老款的更加宽大，整体风格看起来更有冲击力。  \u3000\u300017寸的豪华型铝合金轮毂，很有SUV的年轻动感气息。让原本的商务MPV，多了一丝年轻的朝气。我很喜欢这个大轮毂，是个抢镜的设计。  \u3000\u3000对置式门把手，这种前后门把整合式的设计，造型简单，但也很独特。让整辆车都充满新颖感和高级感。  \u3000\u3000车尾设计虽然不是很性感，但是有点正正方方的，辨识度还是相当高的。57颗LED构成的尾灯，双层展翼造型，视感很好。  \u3000\u3000

In [ ]:
sentences = train_sentences + val_sentences

corpus = []

for sentence in sentences:
  sentence = str(sentence).strip().replace("\u3000","")
  sts =[s for s in re.split(r'[。!?]',sentence) if len(s) >= 10]
  corpus.extend(sts)

In [ ]:
corpus[:10]

['办理了ETC高速浪一下，一出门就遇到阿尔法，100万哪，老款的我坐过也开过，确实好车，但是没感觉值那么多，中国豪人太多',
 ' 走磨合测试定速巡航 父亲没空就带着家里的各位老板们兜风  途中也感受下老板位舒服不，辛苦大老板了，舒适版有个好处，可以不用下车就可以在车内完成换司机的任务，这个对于下雨天啥的，非常实用',
 '  三老板，其实是最大的老板，哭起来，我都要投降 二老板  趁着假装老板的机会好好感受下车内，第2排左侧位置放个纸巾不错 中间没有扶手的好处可以放相机包还有垃圾桶，这个建议大家都要备一个，乱丢垃圾的事我们可不干',
 ' 2排空调面板设计的还不错 扶手也挺好的，多有 电镀处理，而且是亚光的，更显质感',
 '  一路奔袭，这条路上基本无车，风景又美，舒服惬意',
 ' 终于抵达目的地，永泰的葱饼店，这个可是福州第一名的葱饼，高峰期的时候要排队一个小时，今天时候尚早还没开始排，赶紧趁热走一个  途进服务区，拍个造型  这个角度有没很帅 最后大鹏展翅来一个  来回50多公里吧，平均油耗6.7.80公里时速，这个成绩这样的车重应该算是优秀的',
 '  目前使用了200多公里，感受良好，全家一直满意，现在有点后悔没有一步到位上个顶配',
 '算了，走改装之路吧！比较遗憾的是目前还没找到电动后视镜，还有自动刹车也无法加装，其他的配置倒不是很在意',
 '因为手上的两地牌关系，一直都喜欢买跑车开，觉得用了两地牌在国内开，而且完善的赛道设施，可以在赛道上飚车确实在香港是无法想象的 ',
 '  因为年纪渐长，有了家庭，为了能够方便孩子家里出入方便，决定换一台7人保姆车来开，近期埃尔法在国内流行人气指数又超高，于是决定入手这辆适合家庭出行的保姆车，开来看去原厂样式配置的埃尔法就是有点老气的感觉，所以一直在看日本改装大厂对这台车的改装有哪些自己比较喜欢的，结果一看到Wald对这台车的改装后立即就吸引到我了，拿起电话打给中国总代，问问关于这台车的改装方案']

In [ ]:
random.shuffle(sentences)
train_sentences,val_sentences = train_test_split(corpus,test_size=0.05,random_state=42)

In [ ]:
len(train_sentences),len(val_sentences)

(22686, 1195)

In [ ]:
val_sentences[:10]

['全合成的机油是最好的，楼主说的美孚是国际三大巨头之一是可以使用的，博世的机滤真品也是可以的',
 '前排通风和第二排按摩是最有价值的，我平时带司机所以只纠结座椅按摩',
 ' 外观比较大气，紫檀红，个人感觉商用家用都OK      尾部感觉一般   新换的前脸还不错    下面就是内饰了   怎么样，位置空间都还不错吧  大鹏展翅哦  电动后尾箱，带踢脚感应功能，灵敏度还行',
 '环抱式中控台仍然是熟悉的别克风格，酒红色的仿皮材质中控台带来了质感的提升',
 '因此，GL8的顶配车型，Avenir就出现在人们面前',
 '外观优雅、大气，内饰的用料和手感也让我爱不释手，接着又试驾了一下……然后就有了现在的作业！试驾之前，真没想到这么个大家伙还能拥有如此轻快的操作体验，而且越来越觉得它跟自己的气场无比契合，回去就派人去和销售联系签订了订车协议，协议一定，经理便和我联系，一步一步手续等都是经理亲自和我联系办理的，让我觉得还挺与众不同的，最终喜得全新一代GL8 Avenir而归！ 从提车到现在，两个月的时间虽不算久，却也足够让我们更亲近，决定拿出我们这段时间接触的点点滴滴与大家分享一下，希望也能够给关注中的朋友们一点点参考，那就算我这半天的字没有白码了',
 '360°环抱一体式设计，融入了家族“展翼”式风格，形面包面切蕴含张力；渐高式影院布局，带给后排更加宽敞明亮的视野；更贴合人体曲线的贵宾级豪华真皮座椅，支持10项调节功能，超过38道工艺工序、家族式菱形花纹与Avenir标志设计，更平添了几分豪华感，160°的靠背倾斜、60°的腿托支撑，为乘客带来堪称极致的乘坐体验；超大尺寸的全景双天窗，配备100%遮光率的电动遮阳帘，创造出舒适静谧的座舱环境',
 '服务流程比较差比较乱的华熙',
 '   空间方面，全新GL8整车长度达到5203mm，宽度1878mm，高度1811mm，已经是在满足城市常规停车位一把入库的前提下的最大尺寸;后备箱空间正常状态下即拥有521升，最大可达到1650升，可以说是目前国内同级别车型中为数不多的能够在7人满员情况下，后备箱依然拥有相当可观装载能力的大型MPV车型',
 '所以轿车没有考虑，基本就在SUV和MPV中徘徊']

In [4]:
model_name = "hfl/chinese-bert-wwm"
per_device_train_batch_size = 16

save_steps = 1000               #Save model every 1k steps
num_train_epochs = 3            #Number of epochs
use_fp16 = False                #Set to True, if your GPU supports FP16 operations
max_length = 256                #Max length for a text input
do_whole_word_mask = True       #If set to true, whole words are masked
mlm_prob = 0.15                 #Probability that a word is replaced by a [MASK] token

In [9]:
# Load the model
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

output_dir = "output/{}-{}".format(model_name.replace("/", "_"),  datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
print("Save checkpoints to:", output_dir)

Some weights of the model checkpoint at hfl/chinese-bert-wwm were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Save checkpoints to: output/hfl_chinese-bert-wwm-2022-06-12_03-15-16


In [ ]:
pprint(tokenizer("今天天气不错", "我准备出门吃个饭"))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101,
               791,
               1921,
               1921,
               3698,
               679,
               7231,
               102,
               2769,
               1114,
               1906,
               1139,
               7305,
               1391,
               702,
               7649,
               102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
class TokenizedSentencesDataset:
    def __init__(self, sentences, tokenizer, max_length, cache_tokenization=False):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.max_length = max_length
        self.cache_tokenization = cache_tokenization

    def __getitem__(self, item):
        if not self.cache_tokenization:
            return self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)

        if isinstance(self.sentences[item], str):
            self.sentences[item] = self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)
        return self.sentences[item]

    def __len__(self):
        return len(self.sentences)

In [ ]:
train_dataset = TokenizedSentencesDataset(train_sentences, tokenizer, max_length)
dev_dataset = TokenizedSentencesDataset(val_sentences, tokenizer, max_length, cache_tokenization=True) if len(val_sentences) > 0 else None

In [ ]:
##### Training arguments

if do_whole_word_mask:
    data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)
else:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)

In [ ]:
len(train_dataset),len(dev_dataset)

(22686, 1195)

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    evaluation_strategy="steps" if dev_dataset is not None else "no",
    per_device_train_batch_size=per_device_train_batch_size,
    eval_steps=save_steps,
    save_steps=save_steps,
    logging_steps=save_steps,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=use_fp16
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

In [ ]:
print("Save tokenizer to:", output_dir)
tokenizer.save_pretrained(output_dir)

tokenizer config file saved in output/hfl_chinese-bert-wwm-2022-06-11_14-27-03/tokenizer_config.json
Special tokens file saved in output/hfl_chinese-bert-wwm-2022-06-11_14-27-03/special_tokens_map.json


Save tokenizer to: output/hfl_chinese-bert-wwm-2022-06-11_14-27-03


('output/hfl_chinese-bert-wwm-2022-06-11_14-27-03/tokenizer_config.json',
 'output/hfl_chinese-bert-wwm-2022-06-11_14-27-03/special_tokens_map.json',
 'output/hfl_chinese-bert-wwm-2022-06-11_14-27-03/vocab.txt',
 'output/hfl_chinese-bert-wwm-2022-06-11_14-27-03/added_tokens.json',
 'output/hfl_chinese-bert-wwm-2022-06-11_14-27-03/tokenizer.json')

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 22686
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4254


Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
print("Save model to:", output_dir)
model.save_pretrained(output_dir)

print("Training done")

### sliding window
处理长文本 滑动窗口

In [25]:
context = """
单位用车，选车的时候直接略过了gl8、塞纳，考虑过凯路威，试驾一次，不是感觉特别好，减震硬的一B，不喜欢仪表台，好渣！然后在论坛看到威霆，就对威霆有了兴趣，尤其是高顶威霆，外观霸气侧漏，内饰改好了的话也很高端，一点不差劲，接下来就找车吧，这个艰难的，哪哪都加价，没办法加就加吧，谁让咱用车呢，提车改装都是一个人帮我解决的，内心还是很感谢的！外观加了高顶，改了电动侧滑门内饰：航空座椅、折叠沙发床、云格灯、地板、桃木件，全车内饰做了大能隔音，还有吸顶电视，这个不错，加上小米盒子可以直接在线看网络电视，前提是
3be38db244e74768beb2188a0d4ff861,【一路同行】别克GL8 Avenir提车作业,                  本人从商，经常都要与很多商业界的人士往来，形象工程当然也是必不可少滴，拥有一部好车可以让自己出入每个场所都显得体面，这也给人呈现一种实力的象征，在很多的合作领域也更能得到别人对自己实力的认可，可以更自信地去应对自己所做每个策略。当时我是看的大众的迈特威，纠结了两天我还是把定金交了！然后我朋友跟我说全新一代的GL8的豪华商务车那款还不错，比你买的迈特威空间要大，外观看上去更有吸引力！于是我抽了一天时间和朋友一起去4S店看了全新一代GL8豪华商务车。&nbsp;一进店，一位姓鲁的男销售顾问就很热情的迎了上来！在销售顾问介绍完以后觉得比迈特威要好得太多太多了，光看外观就更胜一筹，内饰就更不要说了，我对比了几个配置，我喜欢GL8&nbsp;Avenir私人订制版的这个配置，车身颜色挺吸引人的！而且相同价位别克的GL8&nbsp;Avenir要多了很多很多配置，像膝部气囊（安全很重要）车道偏离、主动刹车（这些高级驾驶辅助系统都是迈特威没有的）以及电动的全景天窗（大爱这个），GL8&nbsp;Avenir的扬声器还是BOSS的！这里必须给个赞！没有对比就没有伤害，没想到GL8&nbsp;Avenir性价比居然这么高，我果断去把迈特威退了，过来订了这款GL8&nbsp;Avenir私人订制版！！订车时听销售顾问说GL8&nbsp;Avenir是量产的，心里还着急有没有车，但是我运气好，销售顾问告知我最新订的一批车昨天刚刚到店，说我今天把手续办完明天就可以提车！按照约定交车时间我来到九城别克4S店提车先来个正面图。订完车回家了解了一下Avenir在法语中寓意着“未来”，是对美好前程的预示！希望拥有Avenir后我在未来的日子里更加美好！前脸是菱形网状，体现不凡的气度，别克盾牌标上面还有一层类似于玻璃材质覆盖在上面，显得更加大气。右前45°相片，双层组合的LED大灯，很犀利！&nbsp;门边还有个Avenir独特的字母标识，更加彰显它的尊贵身份。发动机舱，2.0T的涡轮发动机，点下油门就能够感受它强劲的推背感。驾驶室、看不到抬头显、后续我会追加用车心得、大家敬请期待。第二排VIP座椅站在尾箱处往前看，车内空间很大哦！座椅的头枕上有Avenir专属英文标识这里也有专属英文标识！无时无刻都在衬托着它的尊贵身份。门边的储物空间非常满意！来个侧边45°相片首排座椅中央对开式储物盒网传的认证神奇最后附上一张提车相片，小鲁服务态度很好！也非常专业！大家买车可以找他。……………………………………………………THE&nbsp;&nbsp;&nbsp;END！
"""

question = "小改怡情】分享我的威霆改装高顶~~附大妞"

len(context),len(question)

(1411, 20)

In [26]:
max_length = 1024 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.

In [29]:
len(tokenizer(question, context)["input_ids"])

1309

In [31]:
len(tokenizer(question,
              context, 
              max_length=max_length, 
              truncation="only_second")["input_ids"])

1024

In [32]:
tokenized_example = tokenizer(
    question,
    context,
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [33]:
for k in tokenized_example:
  print(k,tokenized_example[k])

input_ids [[101, 2207, 3121, 2592, 2658, 524, 1146, 775, 2769, 4638, 2014, 7447, 3121, 6163, 7770, 7553, 172, 172, 7353, 1920, 1977, 102, 1296, 855, 4500, 6756, 8024, 6848, 6756, 4638, 3198, 952, 4684, 2970, 4526, 6814, 749, 9101, 8156, 510, 1853, 5287, 8024, 5440, 5991, 6814, 1132, 6662, 2014, 8024, 6407, 7730, 671, 3613, 8024, 679, 3221, 2697, 6230, 4294, 1166, 1962, 8024, 1121, 7448, 4801, 4638, 671, 144, 8024, 679, 1599, 3614, 811, 6134, 1378, 8024, 1962, 3942, 8013, 4197, 1400, 1762, 6389, 1781, 4692, 1168, 2014, 7447, 8024, 2218, 2190, 2014, 7447, 3300, 749, 1069, 6637, 8024, 2215, 1071, 3221, 7770, 7553, 2014, 7447, 8024, 1912, 6225, 7464, 3698, 904, 4026, 8024, 1079, 7652, 3121, 1962, 749, 4638, 6413, 738, 2523, 7770, 4999, 8024, 671, 4157, 679, 2345, 1226, 8024, 2970, 678, 3341, 2218, 2823, 6756, 1416, 8024, 6821, 702, 5680, 7410, 4638, 8024, 1525, 1525, 6963, 1217, 817, 8024, 3766, 1215, 3791, 1217, 2218, 1217, 1416, 8024, 6443, 6375, 1493, 4500, 6756, 1450, 8024, 2990, 6756,

In [34]:
len(tokenized_example['input_ids'])

2

In [35]:
for x in tokenized_example["input_ids"]:
  print(tokenizer.decode(x))

[CLS] 小 改 怡 情 】 分 享 我 的 威 霆 改 装 高 顶 ~ ~ 附 大 妞 [SEP] 单 位 用 车 ， 选 车 的 时 候 直 接 略 过 了 gl8 、 塞 纳 ， 考 虑 过 凯 路 威 ， 试 驾 一 次 ， 不 是 感 觉 特 别 好 ， 减 震 硬 的 一 b ， 不 喜 欢 仪 表 台 ， 好 渣 ！ 然 后 在 论 坛 看 到 威 霆 ， 就 对 威 霆 有 了 兴 趣 ， 尤 其 是 高 顶 威 霆 ， 外 观 霸 气 侧 漏 ， 内 饰 改 好 了 的 话 也 很 高 端 ， 一 点 不 差 劲 ， 接 下 来 就 找 车 吧 ， 这 个 艰 难 的 ， 哪 哪 都 加 价 ， 没 办 法 加 就 加 吧 ， 谁 让 咱 用 车 呢 ， 提 车 改 装 都 是 一 个 人 帮 我 解 决 的 ， 内 心 还 是 很 感 谢 的 ！ 外 观 加 了 高 顶 ， 改 了 电 动 侧 滑 门 内 饰 ： 航 空 座 椅 、 折 叠 沙 发 床 、 云 格 灯 、 地 板 、 桃 木 件 ， 全 车 内 饰 做 了 大 能 隔 音 ， 还 有 吸 顶 电 视 ， 这 个 不 错 ， 加 上 小 米 盒 子 可 以 直 接 在 线 看 网 络 电 视 ， 前 提 是 3be38db244e74768beb2188a0d4ff861, 【 一 路 同 行 】 别 克 gl8 avenir 提 车 作 业, 本 人 从 商 ， 经 常 都 要 与 很 多 商 业 界 的 人 士 往 来 ， 形 象 工 程 当 然 也 是 必 不 可 少 滴 ， 拥 有 一 部 好 车 可 以 让 自 己 出 入 每 个 场 所 都 显 得 体 面 ， 这 也 给 人 呈 现 一 种 实 力 的 象 征 ， 在 很 多 的 合 作 领 域 也 更 能 得 到 别 人 对 自 己 实 力 的 认 可 ， 可 以 更 自 信 地 去 应 对 自 己 所 做 每 个 策 略 。 当 时 我 是 看 的 大 众 的 迈 特 威 ， 纠 结 了 两 天 我 还 是 把 定 金 交 了 ！ 然 后 我 朋 友 跟 我 说 全 新 一 代 的 gl8 的 豪 华 商 务 车 那 款 还 不 错 ， 比 你 买 的 迈 特 威 空 间 要 大 ， 外 观 看 上 去 更 有 